In [22]:
#am
import pandas as pd
from ipynb.fs.full.cleaning_account_dim import check_na_columns
import yfinance as yf
import time
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar

In [23]:
df = pd.read_csv("../data/wrld_stor_tran_fact_20250325.csv")
df.head()

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt,product_amt,product_qty,...,first_purchase_ind,adj_orgn_tran_dt,curr_markup_fee,fcr_amount,fcr_flag,fcr_rate_of_exchange,frgn_curr_code,frgn_tran_amt,us_equiv_amt,posting_date
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1,NaN,NaN,...,N,2024-03-17,0.0,NaN,NaN,NaN,840,47.40,0.0,2024-04-30
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,87.13,0.0,2024-06-27
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,30.00,0.0,2024-08-07
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,32.30,0.0,2024-09-05
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,4.40,0.0,2024-12-31


In [24]:

all_sheets = pd.read_excel('country_codes.xls', sheet_name=None)
numeric_codes = all_sheets['Active']['Unnamed: 3']
alphabetic_codes = all_sheets['Active']['Unnamed: 2']
print(alphabetic_codes)

0                  NaN
1                  NaN
2      Alphabetic Code
3                  AFN
4                  EUR
            ...       
277                XXX
278                XAU
279                XPD
280                XPT
281                XAG
Name: Unnamed: 2, Length: 282, dtype: object


In [25]:

def trading_days_back(start, end):
    
    us_bd = pd.offsets.CustomBusinessDay(holidays=USFederalHolidayCalendar().holidays(start, end))
    return len(pd.date_range(start, end, freq=us_bd)) - 1



In [26]:
codeToConversion = {}
for index, row in df.iterrows():
    code = row['frgn_curr_code']
    #if code == '051':
    #    continue

    if str(code) == '840': ### american Samosa uses USD
        df.loc[index,'us_equiv_amt'] = row['transaction_amt']
        continue

    if code in codeToConversion:
        data = codeToConversion[code]
        idx =  700 - trading_days_back(datetime.datetime.strptime(row['transaction_date'], format_string), datetime.datetime.today())#(datetime.datetime.today() - datetime.datetime.strptime(row['transaction_date'], format_string)).days
        #print(idx)
        conversion = float(data["Open"].iloc[idx].iloc[0])
        df.loc[index,'us_equiv_amt'] = row['transaction_amt'] * conversion
        continue

    index = (numeric_codes == code).idxmax() 
    if index == 0:
        continue
    time.sleep(0.5) #time delay to prevent throtteling from api
    format_string = "%Y-%m-%d"

    if alphabetic_codes.loc[index] == "BAM" or alphabetic_codes.loc[index] == "AMD":
        continue

    try:
        data = yf.download(alphabetic_codes.loc[index] + "USD=x", datetime.datetime.today() - datetime.timedelta(1000), datetime.datetime.today())
        idx =  700 - trading_days_back(datetime.datetime.strptime(row['transaction_date'], format_string), datetime.datetime.today())
        conversion = float(data["Open"].iloc[idx].iloc[0])
    except:
        continue
    df.loc[index,'us_equiv_amt'] = row['transaction_amt'] * conversion

    codeToConversion[code] = data


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

KeyboardInterrupt: 

In [39]:
df

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt,product_amt,product_qty,...,first_purchase_ind,adj_orgn_tran_dt,curr_markup_fee,fcr_amount,fcr_flag,fcr_rate_of_exchange,frgn_curr_code,frgn_tran_amt,us_equiv_amt,posting_date
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,NaN,0,1,NaN,NaN,...,N,2024-03-17,0.0,NaN,NaN,NaN,840,47.40,47.400000,2024-04-30
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,87.13,87.130000,2024-06-27
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,30.00,30.000000,2024-08-07
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,32.30,32.300000,2024-09-05
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,4.40,6.058175,2024-12-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053849,uqlll7MEoZkj3vur,SALE,253,2024-11-12,9.57,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,9.57,9.570000,2024-11-13
1053850,ZGjYc8aETjtytUYR,SALE,253,2025-03-10,16.11,NaN,0,1,NaN,NaN,...,N,NaN,0.0,NaN,NaN,NaN,840,16.11,16.110000,2025-03-11
1053851,4JpGONAzyfmoxi7A,SALE,253,2025-03-20,20.37,NaN,0,1,NaN,NaN,...,N,2025-02-26,0.0,NaN,NaN,NaN,840,20.37,20.370000,2025-03-21
1053852,tUUUlJJZ2ETedT7g,SALE,253,2024-11-21,180.00,NaN,0,1,NaN,NaN,...,N,2024-11-12,0.0,NaN,NaN,NaN,840,180.00,180.000000,2024-11-22


In [ ]:
#{'524', '044', '170', '826', '410', '554', '084', '756', '484', '504', '682', '953', '052', '985', '458', '986', '975', '818', '203', '036', '710', '360', '344', '050', '901', '608', '604', '388', '348', '704', '320', '340', '404', '124', '400', '144', '949', '136', '977', '702', '578', '051', '634', '840', nan, '764', '152', '376', '936', '156', '978', '\\"\\"', '352', '214', '356', '414', '784', '586', '208', '788', '752', '032', '951', '188', '392'}

{'524', '044', '170', '826', '410', '554', '084', '756', '484', '504', '682', '953', '052', '985', '458', '986', '975', '818', '203', '036', '710', '360', '344', '050', '901', '608', '604', '388', '348', '704', '320', '340', '404', '124', '400', '144', '949', '136', '977', '702', '578', '051', '634', '840', nan, '764', '152', '376', '936', '156', '978', '\\"\\"', '352', '214', '356', '414', '784', '586', '208', '788', '752', '032', '951', '188', '392'}


In [16]:
df.shape[0]

1053854

In [40]:
cols_to_replace = ["fcr_flag","fcr_amount","product_amt","product_qty","payment_type","fcr_rate_of_exchange"]

In [41]:
check_na_columns(df)

payment_type              1053854
fcr_rate_of_exchange      1053854
fcr_flag                  1053854
product_amt               1053854
product_qty               1053854
fcr_amount                1053854
adj_orgn_tran_dt          1009290
invoice_nbr                887004
us_equiv_amt                   85
frgn_tran_amt                  85
frgn_curr_code                 85
curr_markup_fee                85
current_account_nbr             0
first_purchase_ind              0
transaction_type                0
transaction_sale_cnt            0
transaction_return_cnt          0
transaction_amt                 0
transaction_date                0
transaction_code                0
posting_date                    0
dtype: int64

In [42]:
df[cols_to_replace].dtypes

fcr_flag                float64
fcr_amount              float64
product_amt             float64
product_qty             float64
payment_type            float64
fcr_rate_of_exchange    float64
dtype: object

In [43]:
# we dont want to lose data, so there will be a placeholder (0) for columns that do not have a value

for col in cols_to_replace:
    df[col] = df[col].fillna(0)
check_na_columns(df)

adj_orgn_tran_dt          1009290
invoice_nbr                887004
us_equiv_amt                   85
frgn_tran_amt                  85
frgn_curr_code                 85
curr_markup_fee                85
first_purchase_ind              0
fcr_rate_of_exchange            0
fcr_flag                        0
fcr_amount                      0
current_account_nbr             0
transaction_type                0
product_qty                     0
product_amt                     0
transaction_sale_cnt            0
transaction_return_cnt          0
payment_type                    0
transaction_amt                 0
transaction_date                0
transaction_code                0
posting_date                    0
dtype: int64

In [21]:
# this causes issues with time parsing
# df["adj_orgn_tran_dt"] = df["adj_orgn_tran_dt"].fillna("No Date")
# check_na_columns(df)

In [44]:
df["invoice_nbr"] = df["invoice_nbr"].fillna("No Invoice Number")
check_na_columns(df)

adj_orgn_tran_dt          1009290
us_equiv_amt                   85
frgn_tran_amt                  85
frgn_curr_code                 85
curr_markup_fee                85
current_account_nbr             0
first_purchase_ind              0
fcr_rate_of_exchange            0
fcr_flag                        0
fcr_amount                      0
invoice_nbr                     0
transaction_type                0
product_qty                     0
product_amt                     0
transaction_sale_cnt            0
transaction_return_cnt          0
payment_type                    0
transaction_amt                 0
transaction_date                0
transaction_code                0
posting_date                    0
dtype: int64

In [45]:
df.head()

,current_account_nbr,transaction_type,transaction_code,transaction_date,transaction_amt,payment_type,transaction_return_cnt,transaction_sale_cnt,product_amt,product_qty,...,first_purchase_ind,adj_orgn_tran_dt,curr_markup_fee,fcr_amount,fcr_flag,fcr_rate_of_exchange,frgn_curr_code,frgn_tran_amt,us_equiv_amt,posting_date
0,8DPC9336FwGf8TPz,SALE,253,2024-04-29,47.40,0.0,0,1,0.0,0.0,...,N,2024-03-17,0.0,0.0,0.0,0.0,840,47.40,47.400000,2024-04-30
1,DRTiAtZURdSfMAB0,SALE,253,2024-06-25,87.13,0.0,0,1,0.0,0.0,...,N,NaN,0.0,0.0,0.0,0.0,840,87.13,87.130000,2024-06-27
2,4y1CqjRDtP3w8Nb5,SALE,253,2024-08-06,30.00,0.0,0,1,0.0,0.0,...,N,NaN,0.0,0.0,0.0,0.0,840,30.00,30.000000,2024-08-07
3,3dPDITyRPlzdq3wf,SALE,253,2024-09-05,32.30,0.0,0,1,0.0,0.0,...,N,NaN,0.0,0.0,0.0,0.0,840,32.30,32.300000,2024-09-05
4,q1mnsZTeAaQH71AF,SALE,253,2024-12-30,4.40,0.0,0,1,0.0,0.0,...,N,NaN,0.0,0.0,0.0,0.0,840,4.40,6.058175,2024-12-31


In [46]:
df["transaction_date"] = pd.to_datetime(df["transaction_date"], errors="coerce")
df["adj_orgn_tran_dt"] = pd.to_datetime(df["adj_orgn_tran_dt"], errors="coerce")
df["posting_date"] = pd.to_datetime(df["posting_date"], errors="coerce")
df.dtypes

current_account_nbr               object
transaction_type                  object
transaction_code                   int64
transaction_date          datetime64[ns]
transaction_amt                  float64
payment_type                     float64
transaction_return_cnt             int64
transaction_sale_cnt               int64
product_amt                      float64
product_qty                      float64
invoice_nbr                       object
first_purchase_ind                object
adj_orgn_tran_dt          datetime64[ns]
curr_markup_fee                  float64
fcr_amount                       float64
fcr_flag                         float64
fcr_rate_of_exchange             float64
frgn_curr_code                    object
frgn_tran_amt                    float64
us_equiv_amt                     float64
posting_date              datetime64[ns]
dtype: object

In [47]:
df.to_csv("../cleaned_data/cleaned_wrld_stor_tran_fact.csv",index=False)